In [99]:
#Multivariate Analysis
#Restricted Cubic Split - 4 knots

#Import libraries
library(ggpubr)
library(dplyr)
library(lme4)
library(ggplot2)
library(rms)
library(tidymv)
library("ggthemes")
library(broom)
library(erer)
library(stringr)
library(tidyverse)
library(stringr)
library(R.utils)
library(data.table)
library(ggsci)
library(broom)
library(mgcv)
library(HRW)
library(mgcViz)

In [100]:
#Restricted analysis (only)

#Read the complete set (no imputation)
UKBB_AG2_m <- fread("~/jupyter/UKBB_AG2_12Jan21.txt", header = TRUE, na.strings=c("",".","NA")) %>% select(f.eid,T2D_status,ALBUMINERIA.0.0,
                                 ESKD.0.0,CKD.0.0,DN.0.0,ALL.0.0,NONESKD.0.0,DNCKD.0.0,
                                 CTRL_DNCKD.0.0,ACR.0.0,EGFR.0.0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,SEX.0.0,
                                 IDEAL_DIET2.0.0,LIFESCORE,AGE.0.0, SES_TDI.0.0,BMI.0.0,EDUYEARS,SBP.0.0,HYP_POS1,STATIN,
                                 WHR.0.0,GRS_WT_IR,GRS_WT_T2DIR,T2D.0.0,
                                 GRS_WT_LIRd2,GRS_WT_LIRt,GRS_WT_LIRq,GRS_WT_LIRf3,GRS_WT_IRd2,
                                 GRS_WT_IRt,GRS_WT_IRq,GRS_WT_IRf3,GRS_WT_IR53d2,GRS_WT_IR53t,
                                 GRS_WT_IR53q,GRS_WT_IR53f3,GRS_WT_T2DIRd2,GRS_WT_T2DIRt,
                                 GRS_WT_T2DIRq,GRS_WT_T2DIRf3,GRS_WT_L5E8IRd2,GRS_WT_L5E8IRt,
                                 GRS_WT_L5E8IRq,GRS_WT_L5E8IRf3,GRS_WT_L1E5IRd2,GRS_WT_L1E5IRt,
                                 GRS_WT_L1E5IRq,GRS_WT_L1E5IRf3)
UKBB_AG2=as.data.frame(UKBB_AG2_m)
dim(UKBB_AG2)
rm(UKBB_AG2_m)

[1] 373487     60

In [101]:
#Dichotomize Outcomes for Logistic Regression

#1_CKD
UKBB_AG2$CKD_only.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD","CKD",NA)),
                levels = c("CKD controls", "CKD"))
#Set the refernece
UKBB_AG2$CKD_only.0.0 <- relevel(UKBB_AG2$CKD_only.0.0, ref = "CKD controls")

#2_CKD Extreme
UKBB_AG2$CKD_ex.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD extreme","CKD extreme",NA)),
                levels = c("CKD controls", "CKD extreme"))
#Set the refernece
UKBB_AG2$CKD_ex.0.0 <- relevel(UKBB_AG2$CKD_ex.0.0, ref = "CKD controls")

#3_Micro
UKBB_AG2$micro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="micro","micro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "micro"))
#Set the reference
UKBB_AG2$micro.0.0 <- relevel(UKBB_AG2$micro.0.0, ref = "normo")

#4_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))
#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#5_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))

#6_ESKD vs. Macro
UKBB_AG2$ESKD_macro.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",NA)),
                levels = c("macro","ESKD"))

#7_DNCKD vs. Control DNCKD
UKBB_AG2$DNCKD2.0.0 <- factor(ifelse(UKBB_AG2$DNCKD.0.0=="yes","DNCKD",
                              ifelse(UKBB_AG2$CTRL_DNCKD.0.0=="yes","DNCKD Control",NA)),
                levels = c("DNCKD Control","DNCKD"))

#8_ESKD vs. Normo, Macro, Micro
UKBB_AG2$ESKD_Albu.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0 %in% c("normo","macro","micro"),"albu",NA)),
                levels = c("albu","ESKD"))

#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#Summarize Counts of Disease Outcomes
table(UKBB_AG2$CKD_only.0.0) #1
table(UKBB_AG2$CKD_ex.0.0) #2
table(UKBB_AG2$micro.0.0) #3
table(UKBB_AG2$macro.0.0) #4
table(UKBB_AG2$ESKD.0.0) #5
table(UKBB_AG2$DN.0.0) #6
table(UKBB_AG2$ALL.0.0) #7
table(UKBB_AG2$ESKD.0.0) #8
table(UKBB_AG2$ESKD_macro.0.0) #8
table(UKBB_AG2$ESKD_Albu.0.0) #9
table(UKBB_AG2$DNCKD2.0.0) #10


CKD controls          CKD 
      349669         6108 


CKD controls  CKD extreme 
      349669          984 


 normo  micro 
348496  14070 


 normo  macro 
348496   1120 


    no    yes 
356332    447 

...


    no    yes 
332345  15439 


    no    yes 
356332    447 


macro  ESKD 
  963   447 


  albu   ESKD 
346560    447 


DNCKD Control         DNCKD 
       326513           645 

In [102]:
#Tabulate results
#input: model, exposure, outcome, indicate if non-linear term present
tab_gam <- function(m1,exposure,outcome,nonlinear) {
    res_sum <- data.frame(init=1)
    if (nonlinear==FALSE) {
        df <- data.frame(summary(m1)[c(1,2)]) #p.coef, se, p.pv
        res_sum <- df[which(rownames(df)==paste0(exposure)),] #keep only exposure term
        res_sum$lowci <- exp(res_sum$p.coef-1.96*res_sum$se)
        res_sum$upci <- exp(res_sum$p.coef+1.96*res_sum$se)
        res_sum$estimate <- exp(res_sum$p.coef)
        res_sum$nonlinear_chi.sq <- NA
        res_sum$nonlinear_s.pv <- NA
        res_sum[,"k"] <- NA
        res_sum$edf <- NA
        res_sum[,"k-index"] <- NA
        res_sum[,"p-value"] <- NA
        
    } else if (nonlinear==TRUE) {
        res_sum$lowci <- NA
        res_sum$upci <- NA
        res_sum$estimate <- NA
        res_sum$nonlinear_chi.sq <- summary(m1)$chi.sq
        res_sum$nonlinear_s.pv <- summary(m1)$s.pv
        gam_check <- k.check(m1)
        colnames(gam_check)[1] <- "k"
        res_sum <- cbind(res_sum,gam_check)
    }
    
    if (nonlinear==FALSE) {
        res_sum <- res_sum %>% mutate_if(is.numeric, round, digits=2)
        p.pv <- data.frame(summary(m1)[4])
        res_sum$p.pv <- p.pv[which(rownames(p.pv)==paste0(exposure)),]
    } else if (nonlinear==TRUE) {
        res_sum$p.pv <- NA
    }
    res_sum$ci <- paste0(res_sum$estimate," (",res_sum$lowci,"-",res_sum$upci,")")
    res_sum$exposure <- exposure
    res_sum$model <- outcome
    res_sum$n <- summary(m1)$n
    res_sum$AIC <- AIC(m1)
    res_sum <- res_sum %>% select(exposure,model,ci,p.pv,n,AIC,nonlinear_chi.sq,nonlinear_s.pv,"k",
                                 edf,"k-index","p-value")
    
    res_sum2 <- res_sum
    rm(res_sum)
return(res_sum2)}

#GAM function
gam_bin <- function(data,exposure,outcome) {
    exposure_quo <- enquo(exposure)
    outcome_quo <- enquo(outcome)
    
    #limit to complete cases
    data <- data %>% select((!!exposure_quo),(!!outcome_quo),AGE.0.0,SEX.0.0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
    data <-data[complete.cases(data),]

    #linear
    fmla <- as.formula(paste0(outcome," ~ ",exposure," + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
         PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))
    m1 <- gam(fmla,data=data,family="binomial",method="ML")
    #tabulate
    sum <- tab_gam(m1,exposure,outcome,FALSE)
    sum$knots <- 0
  
    #20 knot
    fmla.3 <- as.formula(paste0(outcome," ~ s(",exposure,",bs='cr'",",k=3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
         PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))
    m2 <- gam(fmla.3,data=data,family="binomial",method="ML")
    #tabulate 
    sum2 <- tab_gam(m2,exposure,outcome,TRUE)
    sum2$knots <- 20
    sum <- rbind(sum,sum2)
    
    #50 knot
    fmla.10 <- as.formula(paste0(outcome," ~ s(",exposure,",bs='cr'",",k=5) + + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
         PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))
    m3 <- gam(fmla.10,data=data,family="binomial",method="ML")
    #tabulate 
    sum3 <- tab_gam(m3,exposure,outcome,TRUE)
    sum3$knots <- 50 
    sum <- rbind(sum,sum3)
    
    #compare models
    model_comp <- data.frame(lrtest(m1,m2))
    model_comp$comp <- 'M1 vs. M2'
    model_comp2 <- data.frame(lrtest(m2,m3))
    model_comp2$comp <- 'M2 vs. M3'
    model_compf <- rbind(model_comp,model_comp2)
    
    return_list <- list(sum,model_compf)
    return(return_list)
}

In [103]:
##Non-Linear Analyses##

#Initilalize distribution
ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
options (datadist ="ddist")

#Prepare outcome variables for analysis (set appropriate ref level)
UKBB_AG2$CKD_only.0.0 <- relevel(as.factor(UKBB_AG2$CKD_only.0.0),"CKD controls")
UKBB_AG2$micro.0.0 <- relevel(as.factor(UKBB_AG2$micro.0.0),"normo")
UKBB_AG2$ALL.0.0 <- relevel(as.factor(UKBB_AG2$ALL.0.0),"no")
UKBB_AG2$DNCKD2.0.0 <- relevel(as.factor(UKBB_AG2$DNCKD2.0.0),"DNCKD Control")
UKBB_AG2$DN.0.0 <- relevel(as.factor(UKBB_AG2$DN.0.0),"no")
UKBB_AG2$macro.0.0 <- relevel(as.factor(UKBB_AG2$macro.0.0),"normo")
UKBB_AG2$CKD_ex.0.0 <- relevel(as.factor(UKBB_AG2$CKD_ex.0.0),"CKD controls")
#Stratify analyses
T2D <- UKBB_AG2 %>% filter(T2D.0.0==1)
ND <- UKBB_AG2 %>% filter(T2D.0.0==0)
summary(T2D$GRS_WT_IR)
summary(ND$GRS_WT_IR)
#Summarize Quantiles
quantile(T2D$'GRS_WT_IR', probs=c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))
quantile(ND$'GRS_WT_IR', probs=c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))
#.29 (10%) 0.372 (35%) 0.42175 (65) 0.503 (95)
quantile(T2D$'GRS_WT_IR', probs=c(0.01,0.05,0.35,0.65,0.95,0.99)) 
quantile(ND$'GRS_WT_IR', probs=c(0.01,0.05,0.35,0.65,0.95,0.99)) 

##IR-PRS##

##T2D Analyses##
##T2D - CKD##
CKD_T2list <- gam_bin(T2D,"GRS_WT_IR","CKD_only.0.0")

CKD_T2list

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1710  0.3529  0.3970  0.3967  0.4400  0.6500 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0910  0.3440  0.3880  0.3882  0.4320  0.7020 

10%       20%       30%       40%       50%       60%       70%       80% 
0.3130000 0.3420000 0.3621568 0.3800000 0.3970000 0.4130000 0.4309988 0.4510000 
      90%      100% 
0.4791686 0.6500000

10%   20%   30%   40%   50%   60%   70%   80%   90%  100% 
0.305 0.333 0.354 0.371 0.388 0.404 0.422 0.443 0.472 0.702

1%        5%       35%       65%       95%       99% 
0.2479072 0.2900000 0.3720000 0.4217536 0.5030000 0.5489488

1%        5%       35%       65%       95%       99% 
0.2380784 0.2820000 0.3630000 0.4130000 0.4960000 0.5410000

[[1]]
               exposure        model              ci      p.pv     n      AIC
1             GRS_WT_IR CKD_only.0.0 1.74 (0.56-5.4) 0.3371289 20120 6143.441
s(GRS_WT_IR)  GRS_WT_IR CKD_only.0.0      NA (NA-NA)        NA 20120 6143.707
s(GRS_WT_IR)1 GRS_WT_IR CKD_only.0.0      NA (NA-NA)        NA 20120 6143.445
              nonlinear_chi.sq nonlinear_s.pv  k      edf   k-index p-value
1                           NA             NA NA       NA        NA      NA
s(GRS_WT_IR)         1.6914126      0.4585113  2 1.346159 0.9209344   0.715
s(GRS_WT_IR)1        0.9348434      0.3406720  4 1.006874 0.9397898   0.975
              knots
1                 0
s(GRS_WT_IR)     20
s(GRS_WT_IR)1    50

[[2]]
       X.Df    LogLik         Df     Chisq Pr..Chisq.      comp
1  14.00000 -3057.721         NA        NA         NA M1 vs. M2
2  14.57249 -3057.281  0.5724925 0.8793149  0.3483894 M1 vs. M2
11 14.57249 -3057.281         NA        NA         NA M2 vs. M3
21 14.01372 -3057.709 -0.5587760 0.8550551  0.3551262 M2 vs. M3

In [ ]:
##Non-Linear Analyses##

#Initilalize distribution
ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
options (datadist ="ddist")

#Prepare outcome variables for analysis (set appropriate ref level)
UKBB_AG2$CKD_only.0.0 <- relevel(as.factor(UKBB_AG2$CKD_only.0.0),"CKD controls")
UKBB_AG2$micro.0.0 <- relevel(as.factor(UKBB_AG2$micro.0.0),"normo")
UKBB_AG2$ALL.0.0 <- relevel(as.factor(UKBB_AG2$ALL.0.0),"no")
UKBB_AG2$DNCKD2.0.0 <- relevel(as.factor(UKBB_AG2$DNCKD2.0.0),"DNCKD Control")
UKBB_AG2$DN.0.0 <- relevel(as.factor(UKBB_AG2$DN.0.0),"no")
UKBB_AG2$macro.0.0 <- relevel(as.factor(UKBB_AG2$macro.0.0),"normo")
UKBB_AG2$CKD_ex.0.0 <- relevel(as.factor(UKBB_AG2$CKD_ex.0.0),"CKD controls")
#Stratify analyses
T2D <- UKBB_AG2 %>% filter(T2D.0.0==1)
ND <- UKBB_AG2 %>% filter(T2D.0.0==0)
summary(T2D$GRS_WT_IR)
summary(ND$GRS_WT_IR)
#Summarize Quantiles
quantile(T2D$'GRS_WT_IR', probs=c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))
quantile(ND$'GRS_WT_IR', probs=c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))
#.29 (10%) 0.372 (35%) 0.42175 (65) 0.503 (95)
quantile(T2D$'GRS_WT_IR', probs=c(0.01,0.05,0.35,0.65,0.95,0.99)) 
quantile(ND$'GRS_WT_IR', probs=c(0.01,0.05,0.35,0.65,0.95,0.99)) 

##IR-PRS##

##T2D Analyses##
##T2D - CKD##
CKD_T2list <- gam_bin(T2D,"GRS_WT_IR","CKD_only.0.0")
##T2D - Micro##
Micro_T2list <- gam_bin(T2D,"GRS_WT_IR","micro.0.0")
##T2D - All##
All_T2list <- gam_bin(T2D,"GRS_WT_IR","ALL.0.0")
##T2D - DN-CKD##
DNCKD_T2list <- gam_bin(T2D,"GRS_WT_IR","DNCKD2.0.0")
##T2D - DN##
DN_T2list <- gam_bin(T2D,"GRS_WT_IR","DN.0.0")
##T2D - Macro##
Macro_T2list <- gam_bin(T2D,"GRS_WT_IR","macro.0.0")
##T2D - CKD Extreme##
CKDEX_T2list <- gam_bin(T2D,"GRS_WT_IR","CKD_ex.0.0")

##ND Analyses##
##ND - CKD##
CKD_NDlist <- gam_bin(ND,"GRS_WT_IR","CKD_only.0.0")
##ND - Micro##
Micro_NDlist <- gam_bin(ND,"GRS_WT_IR","micro.0.0")
##ND - All##
All_NDlist <- gam_bin(ND,"GRS_WT_IR","ALL.0.0")
##ND - DN-CKD##
DNCKD_NDlist <- gam_bin(ND,"GRS_WT_IR","DNCKD2.0.0")
##ND - DN##
DN_NDlist <- gam_bin(ND,"GRS_WT_IR","DN.0.0")
##ND - Macro##
Macro_NDlist <- gam_bin(ND,"GRS_WT_IR","macro.0.0")
##ND - CKD Extreme##
CKDEX_NDlist <- gam_bin(ND,"GRS_WT_IR","CKD_ex.0.0")

##GRS_PRS##

##T2D Analyses##
##T2D - CKD##
CKD_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","CKD_only.0.0")
##T2D - Micro##
Micro_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","micro.0.0")
##T2D - All##
All_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","ALL.0.0")
##T2D - DN-CKD##
DNCKD_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","DNCKD2.0.0")
##T2D - DN##
DN_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","DN.0.0")
##T2D - Macro##
Macro_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","macro.0.0")
##T2D - CKD Extreme##
CKDEX_T2list2 <- gam_bin(T2D,"GRS_WT_T2DIR","CKD_ex.0.0")

##ND Analyses##
##ND - CKD##
CKD_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","CKD_only.0.0")
##ND - Micro##
Micro_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","micro.0.0")
##ND - All##
All_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","ALL.0.0")
##ND - DN-CKD##
DNCKD_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","DNCKD2.0.0")
##ND - DN##
DN_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","DN.0.0")
##ND - Macro##
Macro_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","macro.0.0")
##ND - CKD Extreme##
CKDEX_NDlist2 <- gam_bin(ND,"GRS_WT_T2DIR","CKD_ex.0.0")

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1710  0.3529  0.3970  0.3967  0.4400  0.6500 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0910  0.3440  0.3880  0.3882  0.4320  0.7020 

10%       20%       30%       40%       50%       60%       70%       80% 
0.3130000 0.3420000 0.3621568 0.3800000 0.3970000 0.4130000 0.4309988 0.4510000 
      90%      100% 
0.4791686 0.6500000

10%   20%   30%   40%   50%   60%   70%   80%   90%  100% 
0.305 0.333 0.354 0.371 0.388 0.404 0.422 0.443 0.472 0.702

1%        5%       35%       65%       95%       99% 
0.2479072 0.2900000 0.3720000 0.4217536 0.5030000 0.5489488

1%        5%       35%       65%       95%       99% 
0.2380784 0.2820000 0.3630000 0.4130000 0.4960000 0.5410000

In [ ]:
##Summary for excel

#results
#take all T2D 
T2D_summary <- rbind(CKD_T2list[[1]],Micro_T2list[[1]],
                    All_T2list[[1]],DNCKD_T2list[[1]],
                    DN_T2list[[1]],Macro_T2list[[1]],
                    CKDEX_T2list[[1]])
T2D_summary$strata <- "T2D"
T2D_summary2 <- rbind(CKD_T2list2[[1]],Micro_T2list2[[1]],
                    All_T2list2[[1]],DNCKD_T2list2[[1]],
                    DN_T2list2[[1]],Macro_T2list2[[1]],
                    CKDEX_T2list2[[1]])
T2D_summary2$strata <- "T2D"

#take all ND
ND_summary <- rbind(CKD_NDlist[[1]],Micro_NDlist[[1]],
                    All_NDlist[[1]],DNCKD_NDlist[[1]],
                    DN_NDlist[[1]],Macro_NDlist[[1]],
                    CKDEX_NDlist[[1]])
ND_summary$strata <- "ND"
ND_summary2 <- rbind(CKD_NDlist2[[1]],Micro_NDlist2[[1]],
                    All_NDlist2[[1]],DNCKD_NDlist2[[1]],
                    DN_NDlist2[[1]],Macro_NDlist2[[1]],
                    CKDEX_NDlist2[[1]])
ND_summary2$strata <- "ND"

#model comparisons
T2D_comp <- rbind(CKD_T2list[[2]],Micro_T2list[[2]],
                    All_T2list[[2]],DNCKD_T2list[[2]],
                    DN_T2list[[2]],Macro_T2list[[2]],
                    CKDEX_T2list[[2]])
T2D_comp$strata <- "T2D,IR-PRS"
T2D_comp2 <- rbind(CKD_T2list2[[2]],Micro_T2list2[[2]],
                    All_T2list2[[2]],DNCKD_T2list2[[2]],
                    DN_T2list2[[2]],Macro_T2list2[[2]],
                    CKDEX_T2list2[[2]])
T2D_comp2$strata <- "T2D,T2D-PRS"

#take all ND
ND_comp <- rbind(CKD_NDlist[[2]],Micro_NDlist[[2]],
                    All_NDlist[[2]],DNCKD_NDlist[[2]],
                    DN_NDlist[[2]],Macro_NDlist[[2]],
                    CKDEX_NDlist[[2]])
ND_comp$strata <- "ND,IR-PRS"
ND_comp2 <- rbind(CKD_NDlist2[[2]],Micro_NDlist2[[2]],
                    All_NDlist2[[2]],DNCKD_NDlist2[[2]],
                    DN_NDlist2[[2]],Macro_NDlist2[[2]],
                    CKDEX_NDlist2[[2]])
ND_comp2$strata <- "ND,T2D-PRS"

##export
model_results <- rbind(T2D_summary,T2D_summary2,ND_summary,ND_summary2)
model_comp <- rbind(T2D_comp,T2D_comp2,ND_comp,ND_comp2)
filename <- paste0('/cellar/users/agarduno/jupyter/Analysis/','3_Multivariate_Analysis-GAM_results',Sys.Date(),'.txt')
filename2 <- paste0('/cellar/users/agarduno/jupyter/Analysis/','3_Multivariate_Analysis-GAM_modelcomp',Sys.Date(),'.txt')
write.csv(model_results,filename)
write.csv(model_comp,filename2)

In [ ]:
#Nonlinear plots of interest

#DN, IR-PRS, T2D
# library(voxel)
# fmla <- as.formula(paste0("DN.0.0"," ~ ","s(GRS_WT_IR)"," + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
#          PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))
# m1 <- gam::gam(fmla,data=T2D,family="binomial")
# summary(m1)
# plot(m1)[1]

#Macro, IR-PRS, T2D
# library(voxel)
# fmla <- as.formula(paste0("macro.0.0"," ~ ","s(GRS_WT_IR)"," + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
#          PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))
# m1 <- gam::gam(fmla,data=T2D,family="binomial")
# plot(m1)[1]

In [ ]:
fmla <- as.formula(paste0("macro.0.0"," ~ ","s(GRS_WT_IR,k=3)"," + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
         PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))

m1<- mgcv::gam(fmla,data=T2D,family="binomial",method="ML")
summary(m1)
AIC(m1)

fmla <- as.formula(paste0("macro.0.0"," ~ ","s(GRS_WT_IR,k=4)"," + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + 
         PC5 + PC6 + PC7 + PC8 + PC9 + PC10"))

m1<- mgcv::gam(fmla,data=T2D,family="binomial",method="ML")
summary(m1)
AIC(m1)

In [ ]:
#ARCHIVE CODE

#T2D - MICRO 
#Archive: RCS version, retire (07.15.21)

# M1 <- Glm(micro.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# rms::specs(M1,long=TRUE)
# lrm(formula = micro.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of Microalbuminuria") + xlim(0.248,0.548)

#.29 0.372 0.42175 0.503

##ALL
#T2D - ALL
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")

# T2D <- UKBB_AG2 %>% filter(T2D_status==1)
# summary(T2D$GRS_WT_IR)
# UKBB_AG2$ALL.0.0 <- relevel(as.factor(UKBB_AG2$ALL.0.0),"no")
# M1 <- Glm(ALL.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# rms::specs(M1,long=TRUE)
# lrm(formula = CKD_only.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of All Albuminuria") + xlim(0.248,0.548)

#.29 0.372 0.42175 0.503
#LR chi-sq (5 knots): 361.48

#DN-CKD
#DNCKD2.0.0 - ALL (not linear or non-linear)
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")

# T2D <- UKBB_AG2 %>% filter(T2D_status==1)
# summary(T2D$GRS_WT_IR)
# UKBB_AG2$DNCKD2.0.0 <- relevel(as.factor(UKBB_AG2$DNCKD2.0.0),"DNCKD Control")
# M1 <- Glm(DNCKD2.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# rms::specs(M1,long=TRUE)
# lrm(formula = DNCKD2.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of All Albuminuria") + xlim(0.248,0.548)

#.29 0.372 0.42175 0.503
#LR chi-sq (5 knots): 361.48

##DN
#DN (not linear or non-linear)
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")

# T2D <- UKBB_AG2 %>% filter(T2D_status==1)
# summary(T2D$GRS_WT_IR)
# UKBB_AG2$DN.0.0 <- relevel(as.factor(UKBB_AG2$DN.0.0),"no")
# M1 <- Glm(DN.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# rms::specs(M1,long=TRUE)
# lrm(formula = DN.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of Diabetic Nephropathy") + xlim(0.248,0.548)

#.29 0.372 0.42175 0.503
#LR chi-sq (5 knots): 361.48

##Macro
#Macro
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")

# T2D <- UKBB_AG2 %>% filter(T2D_status==1)
# summary(T2D$GRS_WT_IR)
# UKBB_AG2$macro.0.0 <- relevel(as.factor(UKBB_AG2$macro.0.0),"normo")
# M1 <- Glm(macro.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# rms::specs(M1,long=TRUE)
# lrm(formula = macro.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of Macroalbumineria") + xlim(0.248,0.548)

#.29 0.372 0.42175 0.503
#LR chi-sq (5 knots): 361.48

#CKD extreme - ns
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")

# T2D <- UKBB_AG2 %>% filter(T2D_status==1)
# summary(T2D$GRS_WT_IR)
# UKBB_AG2$CKD_ex.0.0 <- relevel(as.factor(UKBB_AG2$CKD_ex.0.0),"CKD controls")
# M1 <- Glm(CKD_ex.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# rms::specs(M1,long=TRUE)
# lrm(formula = CKD_ex.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of Macroalbumineria") + xlim(0.248,0.548)

#.29 0.372 0.42175 0.503
#LR chi-sq (5 knots): 361.48

#ND - Micro (linear)
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(micro.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# M1 <- Glm(micro.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# lrm(formula = micro.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of Micro") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - ALL (linear)
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(ALL.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# M1 <- Glm(ALL.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# lrm(formula = micro.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of All") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - ALL (linear)
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(ALL.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# M1 <- Glm(ALL.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# lrm(formula = micro.0.0 ~ rcs(GRS_WT_IR,4) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of All") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - Macro - ns
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(macro.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# anova(M1)
# lrm(formula = macro.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of CKD") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - Macro - ns
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(macro.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# anova(M1)
# lrm(formula = macro.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of CKD") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - CKD - ns
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(CKD_only.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# anova(M1)
# lrm(formula = CKD_only.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of CKD") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - CKD Extreme - ns
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(CKD_ex.0.0 ~ GRS_WT_IR + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# anova(M1)
# lrm(formula = CKD_ex.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of CKD") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #ND - DN
# ddist <- datadist(UKBB_AG2[,c('AGE.0.0','GRS_WT_IR','SEX.0.0','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10')])
# options (datadist ="ddist")
# T2D <- UKBB_AG2 %>% filter(T2D_status==0)
# summary(T2D$GRS_WT_IR)
# M1 <- Glm(DN.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D, binomial(link="logit")) 
# anova(M1)
# #nonlinear not significant
# rms::specs(M1,long=TRUE)
# anova(M1)
# lrm(formula = DN.0.0 ~ rcs(GRS_WT_IR,3) + AGE.0.0 + SEX.0.0 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
# data = T2D)
# xx <- Predict(M1,GRS_WT_IR)
# ggplot(data=xx) + geom_line(aes(x=GRS_WT_IR,y = yhat)) + theme_classic() + xlab("IR-PRS") + ylab("Log Odds of CKD") + xlim(0.228,0.541)

# #.29 0.372 0.42175 0.503

# #Old Code (Update - initial analysis was not stratified)
# RESULTS_CONT <- data.frame()
# RESULTS_OR <- data.frame()

# ##MODEL APPROACH #1 - Three Risk Groups

# #Previously evaluated non-linearity in prior section#
# #Current section evalautes the continuous form:
# RESULTS_CONT <- data.frame()
# RESULTS_OR <- data.frame()

# for (ii in c('GRS_WT_IR','GRS_RAW_T2DIR')) {
#     for(kk in c('Model1','Model2','Model3','Model4','Model5','Model6')){
#           for (jj in c('relevel(as.factor(CKD_only.0.0),"CKD controls")','relevel(as.factor(CKD_ex.0.0),"CKD controls")',
#                        'relevel(as.factor(micro.0.0),"normo")','relevel(as.factor(macro.0.0),"normo")',
#                        'relevel(as.factor(ESKD.0.0),"no")',
#                        'relevel(as.factor(DN.0.0),"no")','relevel(as.factor(ALL.0.0),"no")',
#                        'relevel(as.factor(ESKD_macro.0.0),"macro")',
#                        'relevel(as.factor(DNCKD2.0.0),"DNCKD Control")',
#                        'relevel(as.factor(ESKD_Albu.0.0),"albu")')) {
              
#             #Used across formulas
#         term <- ii   
              
#         #Model 1 - Age, gender, PCI
#         if(kk == "Model1"){
            
#         fmla <- as.formula(paste0(jj," ~ ", "rcs(",term,", 4)", "+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"), env = environment()) }
#         if(kk == "Model2"){
            
#         fmla <- as.formula(paste0(jj," ~ ", "rcs(",term,", 4)","+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
#                                         + EDUYEARS + SES_TDI.0.0"), env = environment()) }
          
#         if(kk == "Model3"){
            
#         fmla <- as.formula(paste0(jj," ~ ", "rcs(",term,", 4)","+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
#                                         + EDUYEARS + SES_TDI.0.0 + BMI.0.0"), env = environment()) }
        
#         if(kk == "Model4"){
            
#         fmla <- as.formula(paste0(jj," ~ ", "rcs(",term,", 4)","+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
#                                         + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + HYP_POS1"), env = environment()) }
              
#         if(kk == "Model5"){
            
#         fmla <- as.formula(paste0(jj," ~ ", "rcs(",term,", 4)","+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
#                                         + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + STATIN"), env = environment()) }
              
#         if(kk == "Model6"){
            
#         fmla <- as.formula(paste0(jj," ~ ", "rcs(",term,", 4)","+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
#                                         + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + HYP_POS1 + STATIN"), env = environment()) }
              
#         M1 <- glm(fmla, data=UKBB_AG2, binomial(link="logit"))                                                                                
#         M1_2 <-  M1 %>% summary()
#         M1_3 <- anova(M1,test="LRT")
        
#         #Model 1 - Sub-groups (same model)
#         #Type 2 Diabetics
#         M1_T2D <- UKBB_AG2 %>% glm(formula=fmla,family=binomial(link="logit"))                                                                                
#         M1_T2D2 <- M1_T2D  %>% summary()
#         M1_T2D3 <- anova(M1_T2D ,test="LRT")
            
#         #Non-Diabetes
#         M1_ND <- UKBB_AG2 %>% glm(formula=fmla, family=binomial(link="logit"))                                                                                
#         M1_ND2 <- M1_ND  %>% summary()
#         M1_ND3 <- anova(M1_ND ,test="LRT")
        
#         TEMP<- list(model=kk, var=jj,var2=ii, total=M1_2,t2d=M1_T2D2,nd=M1_ND2,lrt_tot=M1_3,
#                     lrt_t2d=M1_T2D3,lrt_nd=M1_ND3)
                     
#         #Confidence Intervals
#         #Entire Sample
#         M1$dwn_conf <- coefficients(TEMP$total)[,1]-1.96*coefficients(TEMP$total)[,2]
#         M1$up_conf <- coefficients(TEMP$total)[,1]+1.96*coefficients(TEMP$total)[,2]
#         TABLE2 <- round(cbind(beta=coefficients(TEMP$total)[,1],CI=cbind(M1$dwn_conf,M1$up_conf)),3)   
#         #Diabetics
#         M1_T2D$dwn_conf <- coefficients(TEMP$t2d)[,1]-1.96*coefficients(TEMP$t2d)[,2]
#         M1_T2D$up_conf <- coefficients(TEMP$t2d)[,1]+1.96*coefficients(TEMP$t2d)[,2]
#         TABLE2_T2D <- round(cbind(beta=coefficients(TEMP$t2d)[,1],CI=cbind(M1_T2D$dwn_conf,M1_T2D$up_conf)),3)
#         #Non-Diabetics
#         M1_ND2$dwn_conf <- coefficients(TEMP$nd)[,1]-1.96*coefficients(TEMP$nd)[,2]
#         M1_ND2$up_conf <- coefficients(TEMP$nd)[,1]+1.96*coefficients(TEMP$nd)[,2]
#         TABLE2_ND <- round(cbind(beta=coefficients(TEMP$nd)[,1],CI=cbind(M1_ND2$dwn_conf,M1_ND2$up_conf)),3)
            
#         #Summary Statistics
#         OR_CI <- list(model=kk, var=jj,var2=ii,nd_ci=TABLE2_ND,t2d_ci=TABLE2_T2D,all_ci=TABLE2)
#         RESULTS_OR <- c(RESULTS_OR,OR_CI)    
#         #Combined results
#         RESULTS_CONT <- c(RESULTS_CONT,TEMP)
#        }
#     }
# }

# #Total Sample Score

# #Intermediate and High Risk (Three Categories)
# #Variable of interest listed first
# #Only Save Two Coefficients

# #6 = unique number of elements in list of 180 objects
# #4,5,6 = model-based results (glm summary)
# coef_all3 <- data.frame()
# for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

#   #identifiers
#   model_id <- 2 + 9*(i-1) #model id
#   i1 <- 4 + 9*(i-1) #total models
#   i2 <- 5 + 9*(i-1) #t2d models
#   i3 <- 6 + 9*(i-1) #nd models
    
#   #lrt
#   i7 <- 7 + 9*(i-1) #total models
#   i8 <- 8 + 9*(i-1) #t2d models
#   i9 <- 9 + 9*(i-1) #nd models 

    
#   #model 1,2,3
#   term <- i - (i%/%60)*60 + 1 #remainder from subtracting from multiple of 30, depend on the model?
#   i4 <- ifelse(term %in% c(1:10),
#                'Model 1',
#                ifelse(term %in% c(11:20),'Model 2',
#                       ifelse(term %in% c(21:30),'Model 3',
#                             ifelse(term %in% c(31:40),'Model 4',
#                                   ifelse(term %in% c(41:50),'Model 5',
#                                         ifelse(term %in% c(51:60),'Model 6',))))))
  
#   #ENTIRE SAMPLE 
#   #pull coefficients and convert to OR
#   coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[c(2:4),1])
#   id_model <- names(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[c(2:4),1])
#   model_id2 <- RESULTS_CONT[[model_id]]
#   rep_model <- rep(model_id2,dim(coef_all)[1])
#   rep_adj <- rep(i4,dim(coef_all)[1])
  
#   #pull coefficients and calculate 95% CI
#   up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])[,1]+1.96*coefficients(RESULTS_CONT[[i1]])[,2]),2)[2:4])
#   down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])[,1]-1.96*coefficients(RESULTS_CONT[[i1]])[,2]),2)[2:4])
#   #likelihood ratio
#   lrt_total <- round(RESULTS_CONT[[i7]][2,5],4)
#   lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
#   coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
#   coef_all2 <- cbind(coef_all2,coef_all) #coefficients
#   coef_all2 <- cbind(coef_all2,rep_adj)
#   coef_all2 <- cbind(coef_all2,down_coef) #95 CI
#   coef_all2 <- cbind(coef_all2,up_coef) #95 CI
#   coef_all2 <- cbind(coef_all2,lrt_total2)
#   coef_all3 <- rbind(coef_all3,coef_all2)
# }

# #reformat table
# #rename
# names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
# #combine HR and 95% CI
# coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
# coef_all3 <- subset(coef_all3, select = -c(3,5,6))
# #high/low
# coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
# #substring
# coef_all3$sub <- substr(coef_all3$id_model,19,32)
# coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
# #key
# coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
# #spread
# coef_all3 <- subset(coef_all3,select=-c(1))
# #coef_all4 <- spread(coef_all3,key=category,value=combo)


# #T2D Sample Score

# #Total Sample Score

# #6 = unique number of elements in list of 180 objects
# #4,5,6 = model-based results (glm summary)
# coef_all3 <- data.frame()
# for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

#   #identifiers
#   model_id <- 2 + 9*(i-1) #model id
#   i1 <- 4 + 9*(i-1) #total models
#   i2 <- 5 + 9*(i-1) #t2d models
#   i3 <- 6 + 9*(i-1) #nd models
    
#   #lrt
#   i7 <- 7 + 9*(i-1) #total models
#   i8 <- 8 + 9*(i-1) #t2d models
#   i9 <- 9 + 9*(i-1) #nd models 
    
#   #model 1,2,3
#   term <- i - (i%/%60)*60 + 1 #remainder from subtracting from multiple of 30, depend on the model?
#   i4 <- ifelse(term %in% c(1:10),
#                'Model 1',
#                ifelse(term %in% c(11:20),'Model 2',
#                       ifelse(term %in% c(21:30),'Model 3',
#                             ifelse(term %in% c(31:40),'Model 4',
#                                   ifelse(term %in% c(41:50),'Model 5',
#                                         ifelse(term %in% c(51:60),'Model 6',))))))
  
#   #ENTIRE SAMPLE 
#   #pull coefficients and convert to OR
#   coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])),2)[c(2:4),1])
#   id_model <- names(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[c(2:4),1])
#   model_id2 <- RESULTS_CONT[[model_id]]
#   rep_model <- rep(model_id2,dim(coef_all)[1])
#   rep_adj <- rep(i4,dim(coef_all)[1])
  
#   #pull coefficients and calculate 95% CI
#   up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])[,1]+1.96*coefficients(RESULTS_CONT[[i2]])[,2]),2)[2:4])
#   down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])[,1]-1.96*coefficients(RESULTS_CONT[[i2]])[,2]),2)[2:4])
#   #likelihood ratio
#   lrt_total <- round(RESULTS_CONT[[i8]][2,5],3)
#   lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
#   coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
#   coef_all2 <- cbind(coef_all2,coef_all) #coefficients
#   coef_all2 <- cbind(coef_all2,rep_adj)
#   coef_all2 <- cbind(coef_all2,down_coef) #95 CI
#   coef_all2 <- cbind(coef_all2,up_coef) #95 CI
#   coef_all2 <- cbind(coef_all2,lrt_total2)
#   coef_all3 <- rbind(coef_all3,coef_all2)
# }

# #reformat table
# #rename
# names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
# #combine HR and 95% CI
# coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
# coef_all3 <- subset(coef_all3, select = -c(3,5,6))
# #high/low
# coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
# #substring
# coef_all3$sub <- substr(coef_all3$id_model,19,32)
# coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
# #key
# coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
# #spread
# #coef_all3_t2d <- coef_all3
# coef_all3_t2d <- subset(coef_all3,select=-c(1))
# #coef_all4_t2d <- spread(coef_all3,key=category,value=combo)

# #Non-Diabetic Sample Score

# #Total Sample Score

# #6 = unique number of elements in list of 180 objects
# #4,5,6 = model-based results (glm summary)
# coef_all3 <- data.frame()
# for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

#   #identifiers
#   model_id <- 2 + 9*(i-1) #model id
#   i1 <- 4 + 9*(i-1) #total models
#   i2 <- 5 + 9*(i-1) #t2d models
#   i3 <- 6 + 9*(i-1) #nd models
    
#   #lrt
#   i7 <- 7 + 9*(i-1) #total models
#   i8 <- 8 + 9*(i-1) #t2d models
#   i9 <- 9 + 9*(i-1) #nd models 

    
#   #model 1,2,3
#   term <- i - (i%/%60)*60 + 1 #remainder from subtracting from multiple of 30, depend on the model?
#   i4 <- ifelse(term %in% c(1:10),
#                'Model 1',
#                ifelse(term %in% c(11:20),'Model 2',
#                       ifelse(term %in% c(21:30),'Model 3',
#                             ifelse(term %in% c(31:40),'Model 4',
#                                   ifelse(term %in% c(41:50),'Model 5',
#                                         ifelse(term %in% c(51:60),'Model 6',))))))
  
#   #ENTIRE SAMPLE 
#   #pull coefficients and convert to OR
#   coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])),2)[c(2:4),1])
#   id_model <- names(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[c(2:4),1])
#   model_id2 <- RESULTS_CONT[[model_id]]
#   rep_model <- rep(model_id2,dim(coef_all)[1])
#   rep_adj <- rep(i4,dim(coef_all)[1])
  
#   #pull coefficients and calculate 95% CI
#   up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])[,1]+1.96*coefficients(RESULTS_CONT[[i3]])[,2]),2)[2:4])
#   down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])[,1]-1.96*coefficients(RESULTS_CONT[[i3]])[,2]),2)[2:4])
#   #likelihood ratio
#   lrt_total <- round(RESULTS_CONT[[i9]][2,5],3)
#   lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
#   coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
#   coef_all2 <- cbind(coef_all2,coef_all) #coefficients
#   coef_all2 <- cbind(coef_all2,rep_adj)
#   coef_all2 <- cbind(coef_all2,down_coef) #95 CI
#   coef_all2 <- cbind(coef_all2,up_coef) #95 CI
#   coef_all2 <- cbind(coef_all2,lrt_total2)
#   coef_all3 <- rbind(coef_all3,coef_all2)
# }

# #reformat table
# #rename
# names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
# #combine HR and 95% CI
# coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
# coef_all3 <- subset(coef_all3, select = -c(3,5,6))
# #high/low
# coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
# #substring
# coef_all3$sub <- substr(coef_all3$id_model,19,45)
# coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,45)
# #key
# coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
# #spread
# #coef_all3_nd <- coef_all3
# coef_all3_nd <- subset(coef_all3,select=-c(1))
# #coef_all4_nd <- spread(coef_all3,key=category,value=combo)

# #Export the Sheet
# write.csv(coef_all3,'/cellar/users/agarduno/jupyter/Analysis/All_RCS4_12Jan21.txt')
# write.csv(coef_all3_t2d,'/cellar/users/agarduno/jupyter/Analysis/T2D_RCS4_12Jan21.txt')
# write.csv(coef_all3_nd,'/cellar/users/agarduno/jupyter/Analysis/ND_RCS4_12Jan21.txt')